# Introduction to Forwarding Change Validation


Hello and welcome to our Getting to know Batfish series where we showcase key capabilities of Batfish and how they can be used as part of your network automation workflow. 

My name is Matt. 

In this video, I will show you how to validate network forwarding changes using Batfish. 

Network forwarding changes are:
- Common
- Often hard to get right
- Often hard to validate
- Often high-risk

Network engineers frequently make changes that impact forwarding behavior: adding new routes, opening or closing flows, routing traffic through different devices, etc. These changes are often hard to get right and hard to validate. The risks of deploying a bad change can be huge, potentially causing outages or security breaches. 

For these reasons, having a powerful change validation process is crucial for modern networks.

With Batfish, validating changes to your network can be:

This video will show you how Batfish can help get your changes right.
With Batfish, change validation can be:

- Proactive: catch bugs before they hit the network

Proactive, meaning you can validate changes offline, before you deploy them to the network. 
- Any problems will be discovered before they can affect the network, and
- you'll only deploy your changes after you have complete confidence that they will have the intended effects.

- Network-scale: full network behavior of all devices

Change validation with Batfish is also network-scale: Batfish analyzes full end-to-end network behavior, and this allows it to find bugs caused by interactions between devices far away from each other.

- Comprehensive: validate all possible traffic, not just a few probes

It's also comprehensive: Batfish can validate the forwarding behavior for all possible flows. This allows it to find bugs you may never have thought to test. 

All of this makes batfish the ideal tool for your change validation workflow.

# Change Scenario 1: Costing out `core1`
![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/forwarding-change-validation/differential%20forwarding%20network.png)

Let's get started. 

We're going to validate two different changes to the network shown here. 

The network has two host devices connected to `leaf1` -- a webserver `host-www` and a database server `host-db`. 

On the other end, we have two border routers that connect to the external network.

The network is provisioned with failover redundancy for the core routers. All traffic is normally routed through `core1` but will automatically switch to  `core2` in case of a failure or during maintenance.


In this scenario, we want to service `core1` so we need to shift traffic to `core2`. 

We'll implement a change to cost out `core1`, and verify that it does not affect end-to-end reachability. 

In general, we care about three classes of end-to-end traffic: external-to-host, host-to-external, and host-to-host.

For simplicity, I'm going to focus on the external-to-host traffic in this video but a similar process can cover the other classes as well.

In [ ]:
# Import packages and load questions
%run startup.py

# Initialize base snapshot
NETWORK_NAME = "forwarding-change-validation"
BASE_NAME = "base"
BASE_PATH = "networks/forwarding-change-validation/base"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(BASE_PATH, name=BASE_NAME, overwrite=True)

(RUN)

As usual, we'll begin by initializing the current snapshot of our network in Batfish.

Batfish will parse the configuration files, build a model of the network, and automatically computes the RIBs and FIBs.

In [ ]:
# Run traceroutes from border interfaces to hosts
answer = bfq.traceroute(
    startLocation="enter(border.*[GigabitEthernet0/0])",
    headers=HeaderConstraints(dstIps="ofLocation(host.*)")
).answer(snapshot=BASE_NAME)
show(answer.frame())

Now let's simulate the current forwarding behavior using the `traceroute` question.

We want to see how external-to-host traffic is routed, so:
- to specify traffic destined to the hosts, I'll use this `dstIps` `HeaderConstraint` parameter.
  - it specifies that the destination IP should belong to one of the hosts.
- to specify traffic originating from outside the network, we use this `startLocation` parameter.
  - It specifies a set of interfaces to start tracing from.
  - "enter()" says to start tracing as the packets enter the interfaces
  - Within `enter`, we specify the external interfaces of our border routers.

Let's run the question

(RUN)

The results include a flow from each border router, and all possible paths of each flow. The `TraceCount` column indicates how many paths are possible for each flow. As we can see in the `Traces` column, both flows are routed through `core1`, which is all we're concerned with here.  For more information about the `traceroute` question, see our previous video in this series, `Introduction to Forwarding Analysis`.

For now, having confirmed that traffic is being routed through `core1` as expected, let's go ahead with our change to cost it out so that traffic is routed through `core2` instead.

### Initialize the change snapshot


```
$ diff -r base/ change1/
diff -r base/configs/core1.cfg change1/configs/core1.cfg
68c68
<  ip ospf cost 1
---
>  ip ospf cost 500
73c73
<  ip ospf cost 1
---
>  ip ospf cost 500
78c78
<  ip ospf cost 1
---
>  ip ospf cost 500
83c83
<  ip ospf cost 1
---
>  ip ospf cost 500
```

This diff summarizes our change. 

We'll increase the `ospf cost` of each interface on `core1` from `1` to `500`. This will cause the lower-cost routes through `core2` to be preferred.

### Initialize the change snapshot

In [ ]:
CHANGE1_NAME = "change"
CHANGE1_PATH = "networks/forwarding-change-validation/change1"

bf_init_snapshot(CHANGE1_PATH, name=CHANGE1_NAME, overwrite=True)

With these commands I'll initialize a new snapshot that includes this change.

(RUN)

We now have two snapshots of our network -- one before the change and one after the change. Batfish allows us to compare these two versions of the network, which as you'll see is very powerful for change validation.

Note that the change has still not been pushed to the physical network. Remember, we won't push the change until we have full confidence that it's correct.

### Change validation process
1. No traffic is routed through `core1`
2. Outside-to-host traffic is unaffected

Now we're ready to validate our change. 

We'll do this using a two-step process
- Step 1 validates that the change has the intended effect.
  - No outside-to-host traffic is routed through `core1`
- Step 2 validates that it has no unintended effects. 
  - In this case, the change should not affect what traffic can reach the hosts from outside the network.


### Step 1: No outside-to-host traffic is routed through core1

In [ ]:
answer = bfq.reachability(
    pathConstraints=PathConstraints(
        startLocation="enter(border.*[GigabitEthernet0/0])",
        transitLocations="core1"),
    headers=HeaderConstraints(dstIps="ofLocation(host.*)"),
    actions="SUCCESS,FAILURE"
).answer(snapshot=CHANGE1_NAME)
show(answer.frame())

This command performs step 1.

To verify that **no** outside-to-host traffic is routed through `core1`, we search for counterexamples: outside-to-host traffic that *is* routed through `core1`. If no counterexamples are found, we are *guaranteed* that `core1` is never used. 

We use the `reachability` question, which searches for flows that match our search criteria.

As before, we use the same `startLocation` and `dstIps` parameters to specify outside-to-host traffic.

We see two new parameters here:
- The `transitLocations` parameter specifies to search for flows that can transit `core1`. 
- The `actions` parameter specifies to include flows that are dropped as well as those that are successfully delivered (which is the default).

Let's run this query.

(RUN)

Good! The search returned an empty table, which means no counter-examples were found. Since batfish exhaustively searched all outside-to-host flows, we are guaranteed that none can be routed through `core1`. 

This validates that our change has the intended effect.

Next, let's validate that the change has no unintended effects.

### Step 2: Outside-to-host traffic is unaffected.

In [ ]:
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation="enter(border.*[GigabitEthernet0/0])"),
    headers=HeaderConstraints(dstIps="ofLocation(host.*)"),
    actions="SUCCESS"
).answer(
    snapshot=CHANGE1_NAME,
    reference_snapshot=BASE_NAME)
show(answer.frame())

To do that, we'll compare the forwarding behavior of the change snapshot against the original using the `differentialReachability` question. 

This question searches for flows that match the specified criteria in one of the snapshots but not the other.

In this case we're searching for outside-to-host flows that are successfully delivered in one snapshot, and dropped in the other.

If the change is correct, no such flows will be found, because costing out `core1` should have no effect on end-to-end reachability.

(RUN)

Unfortunately, these results show that there is an effect on reachability: some traffic that was being delivered in the reference snapshot (before the change) is being *null routed* in the change snapshot (after the change). This means if we deploy the change now, there will be a loss of connectivity. Thankfully batfish was able to identify that bug before we deployed the change. 

The results include an example flow from each border router. Each flow comes with detailed traces of all the paths it can take through the network, in each snapshot. These traces help us to quickly diagnose the problem: `core2` has an old static route for `2.128.1.1/32` that should have been removed. A similar problem could occur with out-of-date ACLs along the path through `core2`. Batfish would find those problems as well.

In [ ]:
CHANGE1_FIXED_NAME = "change-fixed"
CHANGE1_FIXED_PATH = "networks/forwarding-change-validation/change1-fixed"
bf_init_snapshot(CHANGE1_FIXED_PATH, name=CHANGE1_FIXED_NAME, overwrite=True)

Having identified and diagnosed the problem, we remove the bad static route and load the updated change snapshot into batfish. 

(RUN)

Now let's perform the same validation steps again.

### Step 1 (again): No traffic is routed through core1

In [ ]:
answer = bfq.reachability(
    pathConstraints=PathConstraints(
        startLocation="enter(border.*[GigabitEthernet0/0])",
        transitLocations="core1"),
    headers=HeaderConstraints(
        dstIps="ofLocation(host.*)",
        srcIps="0.0.0.0/0"),
    actions="SUCCESS,FAILURE"
).answer(snapshot=CHANGE1_FIXED_NAME)
show(answer.frame())

Let's run step 1 again, searching for outside-to-host traffic that transits `core1`.

(RUN)

Again, we find no such traffic, so `core1` is still costed-out as intended. 

### Step 2 (again): Outside-to-host traffic is unaffected.

In [1]:
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation="enter(border.*[GigabitEthernet0/0])"),
    headers=HeaderConstraints(dstIps="ofLocation(host.*)", srcIps="0.0.0.0/0")
).answer(
    snapshot=CHANGE1_FIXED_NAME,
    reference_snapshot=BASE_NAME)
show(answer.frame())

NameError: name 'bfq' is not defined

Next we re-run step 2 to check that costing out `core1` now has no impact on outside-to-host traffic.

(RUN)

Success! This time we get an empty table, which means batfish has verified that our change will not affect outside-to-host traffic. We can deploy the change with complete confidence. 

## Recap


Let's recap the steps we took to verify this change:

1. The change had the intended effect.
 - No traffic is routed through `core1`.

In step 1, we verified that the primary intent of the change is achieved: traffic is moved from `core1` to `core2`. We used the `reachability` query to search *all* outside-to-host flows in the network and verify that none will transit `core1` after the change.

2. The change had no unintended effects.
 - All outside-to-host traffic was unaffected.

In step 2 we verified that moving the traffic did not affect outside-to-host traffic. For this, we used the `differentialReachability` question to compare the forwarding behavior of our change snapshot against the original.

# Change Scenario 2: Validating the end-to-end impact of an ACL change
![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/forwarding-change-validation/differential%20forwarding%20network.png)

Now let's validate another change to the same network. Unlike the previous scenario, this time we **do** want to affect end-to-end reachability, and we will verify that our change has the intended effect. As before, we will also verify that it has no *unintended* effects.

For this scenario, imagine that we have developed and tested a new web service on host `host-www`, and are now ready to open it to HTTP traffic from the outside world. 

In [ ]:
# Trace HTTP traffic to host-www from outside the network. 
answer = bfq.traceroute(
    startLocation="enter(border.*[GigabitEthernet0/0])",
    headers=HeaderConstraints(dstIps="ofLocation(host-www)", applications="HTTP")
).answer(snapshot=BASE_NAME)
show(answer.frame())

Let's start by using the `traceroute` question to confirm that the web service is not currenlty accessible from outside the network. The `dstIps` parameter tells traceroute to pick any IP belonging to the web server as the destination IP.

(RUN)

As you can see, the flow is dropped by the ingress ACL `OUTSIDE_TO_INSIDE` on each border router. This is where we'll make our change. 


```
ip access-list extended OUTSIDE_TO_INSIDE
 permit tcp any 2.128.0.0 0.0.1.255 eq ssh
 permit udp any 2.0.0.0 0.255.255.255
 deny ip any any
```

This snippet shows the original ACL definition.

The first line permits SSH traffic to the host subnet. 

```
ip access-list extended OUTSIDE_TO_INSIDE
 permit tcp any 2.128.0.0 0.0.1.255 eq ssh
 permit tcp any 2.128.0.0 0.0.1.255 eq www
 permit udp any 2.0.0.0 0.255.255.255
 deny ip any any
```

Here's the updated version of the ACL.

We've created a similar rule for HTTP.

We permit HTTP to the entire subnet, because there is an ACL on the leaf router that is responsible for the required per-host filtering. 

In [ ]:
# Initialize the change snapshot
CHANGE2_NAME = "change2"
CHANGE2_PATH = "networks/forwarding-change-validation/change2"
bf_init_snapshot(CHANGE2_PATH, name=CHANGE2_NAME, overwrite=True)

# Trace HTTP traffic to host-www from outside the network. 
answer = bfq.traceroute(
    startLocation="enter(border.*[GigabitEthernet0/0])",
    headers=HeaderConstraints(dstIps="ofLocation(host-www)", applications="HTTP")
).answer(snapshot=CHANGE2_NAME)
show(answer.frame())

Let's initialize a new change snapshot that updates both border router ACLs,

and then test it with the same `traceroute` command we used before.

(RUN)

Good. We now see that HTTP traffic can reach `host-www` from outside the network. 

This test gives us some confidence that the change is correct, but we don't yet have complete confidence because we still haven't fully validated it.

If this were a change to a real network, we'd follow the steps outlined in the `Provably Safe ACL and Firewall Changes` video to independently validate the change to each border router ACL. In the interest of time, I'll skip those steps, and proceed to validating the end-to-end network behavior. 

### Change validation process
1. HTTP traffic from outside the network can reach `host-www`.
2. No other traffic is affected.

As before, I'll validate our change in two steps:
1. Step 1 validates that the change has the intended effect.
  - Our web service is now available from outside the network.
1. Step 2 validates that the change has no unintended effects
  - No other traffic is affected.

### Step 1: HTTP traffic from outside the network can reach host-www

In [ ]:
answer = bfq.reachability(
    pathConstraints=PathConstraints(startLocation="enter(border.*[GigabitEthernet0/0])"),
    headers=HeaderConstraints(
        dstIps="ofLocation(host-www)",
        srcIps="0.0.0.0/0",
        applications="HTTP"),
    actions="FAILURE"
).answer(snapshot=CHANGE2_NAME)
show(answer.frame())

The `traceroute` we did earlier demonstrated that after the change, *some* HTTP traffic can reach the webserver from outside the network. However, this doesn't ensure that *all* such traffic can reach it. For that, we use the `reachability` question to search for counterexamples of the requirement: HTTP flows from the outside that *cannot* reach the webserver. 

We set `actions` parameter to `FAILURE` because we're interested in flows that do not reach their destination.

The `srcIps` parameter tells batfish to consider any source IP address in its search. By default, it will consider `srcIps` appropriate for each startLocation.

(RUN) 

Good! Since batfish's comprehensive search found no counterexamples, we are guaranteed that none exist. In other words, the change had the intended effect.

# Step 2: No other traffic is affected

In [ ]:
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation="enter(border.*[GigabitEthernet0/0])"),
    headers=HeaderConstraints(
        dstIps="ofLocation(host-www)",
        srcIps="0.0.0.0/0",
        applications="HTTP"),
    invertSearch=True
).answer(snapshot=CHANGE2_NAME, reference_snapshot=BASE_NAME)
show(answer.frame())

Step 2 is to check that the change has no unintended effects. As before, I'll use the `differentialReachability` question to compare our change snapshot against the original. We want to search for traffic treated differently by the two snapshots other than the traffic we meant to affect. To do this, we'll set the `invertSearch` parameter to `True`, which directs batfish to search outside the specified header space instead of within it.

(RUN)

Unfortunately, our change had a broader impact than we intended, and batfish gives us some counterexample flows.

In this case, we see a flow than can reach the database server after the change, but was denied by the border router ACLs before.

We can also see that this is an HTTP flow, which is supposed to be blocked at the leaf router. 

So we've quickly diagnosed the problem: The leaf router is not properly filtering traffic to the hosts: it permits HTTP to both hosts, rather than just the webserver.

```
$ diff -r change2/ change2-fixed/
diff -r change2/configs/leaf1.cfg change2-fixed/configs/leaf1.cfg
119c119
<  permit tcp any 2.128.0.0 0.0.255.255 eq www
---
>  permit tcp any 2.128.1.0 0.0.0.255 eq www
```

In [ ]:
CHANGE2_FIXED_NAME = "change2-fixed"
CHANGE2_FIXED_PATH = "networks/forwarding-change-validation/change2-fixed"
bf_init_snapshot(CHANGE2_FIXED_PATH, name=CHANGE2_FIXED_NAME, overwrite=True)

This diff summarizes a fix for the buggy ACL on leaf1.

I've already created an updated change snapshot that includes the fix, so let's load it into batfish and begin the validation process again.

Let's load this fixed change snapshot into batfish and begin the validation process again.

(RUN)

### Step 1 (again): HTTP traffic from outside the network can reach host-www

In [ ]:
answer = bfq.reachability(
    pathConstraints=PathConstraints(startLocation="enter(border.*[GigabitEthernet0/0])"),
    headers=HeaderConstraints(dstIps="ofLocation(host-www)", applications="HTTP"),
    actions="FAILURE"
).answer(snapshot=CHANGE2_FIXED_NAME)
show(answer.frame())

First we re-run step 1.

(RUN)

As before, since we did not find any dropped HTTP flows to the webserver, we are guaranteed that all such flows will be delivered successfully. 

### Step 2 (again): No other traffic is affected

In [ ]:
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation="enter(border.*[GigabitEthernet0/0])"),
    headers=HeaderConstraints(dstIps="ofLocation(host-www)", applications="HTTP"),
    invertSearch=True
).answer(snapshot=CHANGE2_FIXED_NAME, reference_snapshot=BASE_NAME)
show(answer.frame())

Next, we re-run step 2.

(RUN)

Great! This time `differentialReachability` returns an empty table, which means no traffic was affected by the changes other than external traffic to our webservice, as intended.

## Recap

Let's recap the steps we took to verify the change in this scenario:

1. The change had the intended effect.
 - HTTP traffic from outside the network can reach host-www

First, we verified that the intent of the change is achieved: our webservice is now reachable from the outside world.

2. The change had no unintended effects.
 - No other external-to-host traffic is affected.

Second, we verified that there were no unintended effects to external-to-host traffic. As in the previous scenario, we used the `differentialReachability` question to compare the forwarding behavior of our change snapshot against the original.

# Thanks!

Want to learn more? Come find us on Slack and Github!

Thanks for watching this demo of network forwarding change validation using Batfish. You've seen how batfish can help provide full-scale, comprehensive change validation proactively, so you can deploy changes to your network with complete confidence.

If you want to learn more about how to use batfish for validating changes to your network, please leave a comment below, or come find us on slack or github. You'll find links in the description.

And don't forget to check out the other videos in the getting to know batfish series.

Thanks!